## Linguo 2.0


In [11]:
# Se corre para descargar los .mp3 del vocabulario de 
import os
import pandas as pd
import re
from gtts import gTTS

# Nombre del archivo CSV
csv_file = "vocabulario.csv"

# Crear la carpeta de audio si no existe
audio_folder = "audio"
os.makedirs(audio_folder, exist_ok=True)

# Cargar el archivo CSV
df = pd.read_csv(csv_file)

# Capitalizar todas las palabras en todas las filas y columnas
df.columns = [col.title() for col in df.columns]
df = df.applymap(lambda x: str(x).strip().title() if isinstance(x, str) else x)
df.to_csv(csv_file, index=False)

# Función para limpiar y formatear el nombre del archivo
def limpiar_nombre(archivo):
    archivo_limpio = re.sub(r'[^a-zA-Z0-9 ]', ' ', archivo).strip()  # Eliminar caracteres no alfanuméricos
    return archivo_limpio.replace(" ", "_").replace("...", "").replace("-", "_").replace("'", "_").replace("?", "")  # Reemplazar espacios con guiones bajos

# Iterar sobre cada palabra en la primera columna y generar el audio
for palabra in df.iloc[:, 0]:  # Tomar solo la primera columna
    if palabra:
        nombre_limpio = limpiar_nombre(palabra)  # Limpiar el nombre del archivo
        audio_path = os.path.join(audio_folder, f"{nombre_limpio}.mp3")
        tts = gTTS(text=palabra, lang="en")  # Generar audio en inglés
        tts.save(audio_path)

print("¡Proceso completado!")


¡Proceso completado!


In [1]:
# con logo mas grande debajo de la ventana
import tkinter as tk
from tkinter import messagebox
import csv
import os


# Función para cargar las palabras y sus traducciones desde un archivo CSV
def cargar_diccionario(csv_file):
    lista_palabras = []
    with open(csv_file, newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Saltar la primera línea (encabezados)
        for row in reader:
            if len(row) == 2:
                lista_palabras.append((row[0].strip(), row[1].strip()))  # Guardar como lista de tuplas (inglés, español)
    return lista_palabras

# Cargar la lista de palabras desde el archivo CSV
diccionario = cargar_diccionario("vocabulario.csv")
indice_actual = 0  # Índice para recorrer la lista de palabras

# Función para ajustar el tamaño de los elementos al cambiar la ventana
def ajustar_tamanio(event=None):
    ancho = tk_root.winfo_width()
    alto = tk_root.winfo_height()
    escala = min(ancho / 800, alto / 600)

    label_palabra.config(font=("Arial", int(44 * escala), "bold"))
    label_traduccion.config(font=("Arial", int(36 * escala)))
    
    for btn in [btn_traducir, btn_pronunciar, btn_anterior, btn_siguiente]:
        btn.config(font=("Arial", int(24 * escala), "bold"),
                   padx=int(20 * escala),
                   pady=int(10 * escala))

def actualizar_botones():
    btn_anterior.config(state=tk.NORMAL if indice_actual > 0 else tk.DISABLED)
    btn_siguiente.config(state=tk.NORMAL if indice_actual < len(diccionario) - 1 else tk.DISABLED)

def mostrar_palabra():
    global indice_actual
    if diccionario:
        palabra_ingles, _ = diccionario[indice_actual]
        label_palabra.config(text=palabra_ingles)
        label_traduccion.config(text="¿  . . .  ?")  # Ocultar la traducción
        actualizar_botones()
        
# Reemplazar espacios con guiones bajos y elimina otros carecteres
def limpiar_nombre(archivo):
    return archivo.replace(" ", "_").replace("...", "").replace("-", "_").replace("'", "_").replace("?", "")  


def mostrar_traduccion():
    global indice_actual
    if diccionario:
        _, traduccion = diccionario[indice_actual]
        label_traduccion.config(text=traduccion)

def pronunciar():
    global indice_actual
    if diccionario:
        palabra_ingles, _ = diccionario[indice_actual]
        palabra_ingles = limpiar_nombre(palabra_ingles)
        print(palabra_ingles)
        audio_file = f"audio/{palabra_ingles}.mp3"
        if os.path.exists(audio_file):
            os.system(f"start {audio_file}" if os.name == "nt" else f"mpg321 {audio_file}")
        else:
            messagebox.showwarning("Audio no encontrado", "No se encontró el archivo de audio para esta palabra.")

def siguiente_palabra():
    global indice_actual
    if indice_actual < len(diccionario) - 1:
        indice_actual += 1
        mostrar_palabra()

def anterior_palabra():
    global indice_actual
    if indice_actual > 0:
        indice_actual -= 1
        mostrar_palabra()

# Configuración de la ventana principal
tk_root = tk.Tk()
tk_root.title("Linguo 2.0")
tk_root.geometry("1000x700")
tk_root.configure(bg="#2E2E2E")
tk_root.iconbitmap("logo/Logohd.ico")


# Crear un frame para el título y el logo
frame_titulo = tk.Frame(tk_root, bg="#444444", height=100)
frame_titulo.pack(fill="x", padx=10, pady=10)

# Título al lado del logo
label_titulo = tk.Label(frame_titulo, text="Linguo 2.0 - Software Para Practicar Vocabulario", font=("Arial", 20, "bold"), fg="white", bg="#444444")
label_titulo.pack(side="left", padx=10)

# Estilo de los botones
btn_style = {
    "font": ("Arial", 12, "bold"),
    "fg": "#FFFFFF",
    "bg": "#444444",
    "activebackground": "#666666",
    "bd": 2,
    "padx": 12,
    "pady": 6,
}

# Etiqueta para mostrar la palabra en inglés
label_palabra = tk.Label(tk_root, text="", font=("Arial", 44, "bold"), fg="#00FF00", bg="#2E2E2E")
label_palabra.pack(pady=20)

# Botón para mostrar la traducción
btn_traducir = tk.Button(tk_root, text="Ver Significado", command=mostrar_traduccion, **btn_style)
btn_traducir.pack(pady=10)

# Etiqueta donde se mostrará la traducción
label_traduccion = tk.Label(tk_root, text="", font=("Arial", 44), fg="#0000FF", bg="#2E2E2E")
label_traduccion.pack(pady=10)

# Botón para reproducir la pronunciación
btn_pronunciar = tk.Button(tk_root, text="Escuchar Pronunciación", command=pronunciar, **btn_style)
btn_pronunciar.pack(pady=10)

# Botones para navegación
frame_navegacion = tk.Frame(tk_root, bg="#2E2E2E")
frame_navegacion.pack(pady=20)

btn_anterior = tk.Button(frame_navegacion, text="⏪ Anterior", command=anterior_palabra, **btn_style)
btn_anterior.grid(row=0, column=0, padx=20)

btn_siguiente = tk.Button(frame_navegacion, text="Siguiente ⏩", command=siguiente_palabra, **btn_style)
btn_siguiente.grid(row=0, column=1, padx=20)

# Mostrar la primera palabra
mostrar_palabra()

# Ajustar tamaño al cambiar la ventana
tk_root.bind("<Configure>", ajustar_tamanio)

# Ejecutar la aplicación
tk_root.mainloop()


TclError: bitmap "logo/Logohd.ico" not defined

In [2]:
#Con ventana de info y reproduce en la ventana y nuevo logo
#Version Final - con reproductor interno
import tkinter as tk
from tkinter import messagebox
import csv
import os
import pygame
from PIL import Image, ImageTk

# Inicializar pygame.mixer para reproducción de audio
pygame.mixer.init()

def cargar_diccionario(csv_file):
    lista_palabras = []
    with open(csv_file, newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            if len(row) == 2:
                lista_palabras.append((row[0].strip(), row[1].strip()))
    return lista_palabras

diccionario = cargar_diccionario("vocabulario.csv")
indice_actual = 0

def mostrar_info():
    info_ventana = tk.Toplevel(tk_root)
    info_ventana.title("Información del Programa")
    info_ventana.geometry("550x400")
    info_ventana.configure(bg="#444444")
    
    # Cargar y mostrar el logo
    logo_img = Image.open("logo/logo.ico")
    logo_img = logo_img.resize((100, 100), Image.LANCZOS)
    logo_tk = ImageTk.PhotoImage(logo_img)
    label_logo = tk.Label(info_ventana, image=logo_tk, bg="#444444")
    label_logo.image = logo_tk
    label_logo.pack(pady=(10, 5))
    
    label_titulo = tk.Label(info_ventana, text="Linguo 2.0", font=("Arial", 14, "bold"), fg="white", bg="#444444")
    label_titulo.pack()
    
    texto_info = (
        "🌟 Bienvenido a Linguo 2.0 🎉🎧 \n"
        "Este software te ayudará a mejorar tu vocabulario en inglés. 🚀📖\n\n"
        "📌 Características:\n"
        "✔️ Muestra palabras en inglés y sus traducciones.\n"
        "✔️ Reproducción de pronunciación en audio.\n"
        "✔️ Interfaz fácil de usar.\n\n"
        "👨‍💻 Creado por: José R. Guignan. 🚀✨ \n\n "
        "💡 ¡Gracias por usar Linguo 2.0! 💙\n"
        
    )
    label_info = tk.Label(info_ventana, text=texto_info, font=("Arial", 11), fg="white", bg="#444444", justify="left")
    label_info.pack(padx=20, pady=10)

def actualizar_botones():
    btn_anterior.config(state=tk.NORMAL if indice_actual > 0 else tk.DISABLED)
    btn_siguiente.config(state=tk.NORMAL if indice_actual < len(diccionario) - 1 else tk.DISABLED)

def mostrar_palabra():
    global indice_actual
    if diccionario:
        palabra_ingles, _ = diccionario[indice_actual]
        label_palabra.config(text=palabra_ingles)
        label_traduccion.config(text="¿  . . .  ?")
        actualizar_botones()

def mostrar_traduccion():
    global indice_actual
    if diccionario:
        _, traduccion = diccionario[indice_actual]
        label_traduccion.config(text=traduccion)

def pronunciar():
    global indice_actual
    if diccionario:
        palabra_ingles, _ = diccionario[indice_actual]
        palabra_ingles = palabra_ingles.replace(" ", "_").replace("...", "").replace("-", "_").replace("'", "_").replace("?", "")
        audio_file = f"audio/{palabra_ingles}.mp3"
        if os.path.exists(audio_file):
            pygame.mixer.music.load(audio_file)
            pygame.mixer.music.play()
        else:
            messagebox.showwarning("Audio no encontrado", "No se encontró el archivo de audio para esta palabra.")

def siguiente_palabra():
    global indice_actual
    if indice_actual < len(diccionario) - 1:
        indice_actual += 1
        mostrar_palabra()

def anterior_palabra():
    global indice_actual
    if indice_actual > 0:
        indice_actual -= 1
        mostrar_palabra()

tk_root = tk.Tk()
tk_root.title("Linguo 2.0")
tk_root.geometry("1000x700")
tk_root.configure(bg="#2E2E2E")
tk_root.iconbitmap("logo/logo.ico")

frame_titulo = tk.Frame(tk_root, bg="#444444", height=100)
frame_titulo.pack(fill="x", padx=10, pady=10)
label_titulo = tk.Label(frame_titulo, text="Linguo 2.0 - Software Para Practicar Vocabulario", font=("Arial", 20, "bold"), fg="white", bg="#444444")
label_titulo.pack(side="left", padx=10)

btn_style = {
    "font": ("Arial", 16, "bold"),
    "fg": "#FFFFFF",
    "bg": "#444444",
    "activebackground": "#666666",
    "bd": 3,
    "padx": 20,
    "pady": 12,
}

label_palabra = tk.Label(tk_root, text="", font=("Arial", 44, "bold"), fg="#00FF00", bg="#2E2E2E")
label_palabra.pack(pady=20)

btn_traducir = tk.Button(tk_root, text="Ver Significado", command=mostrar_traduccion, **btn_style)
btn_traducir.pack(pady=10)

label_traduccion = tk.Label(tk_root, text="", font=("Arial", 44), fg="#0000FF", bg="#2E2E2E")
label_traduccion.pack(pady=10)

btn_pronunciar = tk.Button(tk_root, text="Escuchar Pronunciación", command=pronunciar, **btn_style)
btn_pronunciar.pack(pady=10)

frame_navegacion = tk.Frame(tk_root, bg="#2E2E2E")
frame_navegacion.pack(pady=20)

btn_anterior = tk.Button(frame_navegacion, text="⏪ Anterior", command=anterior_palabra, **btn_style)
btn_anterior.grid(row=0, column=0, padx=20)

btn_siguiente = tk.Button(frame_navegacion, text="Siguiente ⏩", command=siguiente_palabra, **btn_style)
btn_siguiente.grid(row=0, column=1, padx=20)

btn_info = tk.Button(tk_root, text="ℹ️ Info", font=("Arial", 12, "bold"), command=mostrar_info, bg="#555555", fg="white", bd=2)
btn_info.place(relx=1.0, rely=1.0, anchor="se", x=-10, y=-10)

mostrar_palabra()
tk_root.mainloop()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
#Con reproductor de windows - archivo py
import tkinter as tk
from tkinter import messagebox
import csv
import os
import pygame
from PIL import Image, ImageTk

def cargar_diccionario(csv_file):
    lista_palabras = []
    with open(csv_file, newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            if len(row) == 2:
                lista_palabras.append((row[0].strip(), row[1].strip()))
    return lista_palabras

diccionario = cargar_diccionario("vocabulario.csv")
indice_actual = 0

def mostrar_info():
    info_ventana = tk.Toplevel(tk_root)
    info_ventana.title("Información del Programa")
    info_ventana.geometry("550x400")
    info_ventana.configure(bg="#444444")
    
    # Cargar y mostrar el logo
    logo_img = Image.open("logo/logo.ico")
    logo_img = logo_img.resize((100, 100), Image.LANCZOS)
    logo_tk = ImageTk.PhotoImage(logo_img)
    label_logo = tk.Label(info_ventana, image=logo_tk, bg="#444444")
    label_logo.image = logo_tk
    label_logo.pack(pady=(10, 5))
    
    label_titulo = tk.Label(info_ventana, text="Linguo 2.0", font=("Arial", 14, "bold"), fg="white", bg="#444444")
    label_titulo.pack()
    
    texto_info = (
        "🌟 Bienvenido a Linguo 2.0 🎉🎧 \n"
        "Este software te ayudará a mejorar tu vocabulario en inglés. 🚀📖\n\n"
        "📌 Características:\n"
        "✔️ Muestra palabras en inglés y sus traducciones.\n"
        "✔️ Reproducción de pronunciación en audio.\n"
        "✔️ Interfaz fácil de usar.\n\n"
        "👨‍💻 Creado por: José R. Guignan. 🚀✨ \n\n "
        "💡 ¡Gracias por usar Linguo 2.0! 💙\n"
        
    )
    label_info = tk.Label(info_ventana, text=texto_info, font=("Arial", 11), fg="white", bg="#444444", justify="left")
    label_info.pack(padx=20, pady=10)

def actualizar_botones():
    btn_anterior.config(state=tk.NORMAL if indice_actual > 0 else tk.DISABLED)
    btn_siguiente.config(state=tk.NORMAL if indice_actual < len(diccionario) - 1 else tk.DISABLED)

def mostrar_palabra():
    global indice_actual
    if diccionario:
        palabra_ingles, _ = diccionario[indice_actual]
        label_palabra.config(text=palabra_ingles)
        label_traduccion.config(text="¿  . . .  ?")
        actualizar_botones()

def mostrar_traduccion():
    global indice_actual
    if diccionario:
        _, traduccion = diccionario[indice_actual]
        label_traduccion.config(text=traduccion)

def pronunciar():
    global indice_actual
    if diccionario:
        palabra_ingles, _ = diccionario[indice_actual]
        palabra_ingles = palabra_ingles.replace(" ", "_").replace("...", "").replace("-", "_").replace("'", "_").replace("?", "")
        audio_file = f"audio/{palabra_ingles}.mp3"
        if os.path.exists(audio_file):
            os.system(f"start {audio_file}" if os.name == "nt" else f"mpg321 {audio_file}")
        else:
            messagebox.showwarning("Audio no encontrado", "No se encontró el archivo de audio para esta palabra.")

def siguiente_palabra():
    global indice_actual
    if indice_actual < len(diccionario) - 1:
        indice_actual += 1
        mostrar_palabra()

def anterior_palabra():
    global indice_actual
    if indice_actual > 0:
        indice_actual -= 1
        mostrar_palabra()

tk_root = tk.Tk()
tk_root.title("Linguo 2.0")
tk_root.geometry("1000x700")
tk_root.configure(bg="#2E2E2E")
tk_root.iconbitmap("logo/logo.ico")

frame_titulo = tk.Frame(tk_root, bg="#444444", height=100)
frame_titulo.pack(fill="x", padx=10, pady=10)
label_titulo = tk.Label(frame_titulo, text="Linguo 2.0 - Software Para Practicar Vocabulario", font=("Arial", 20, "bold"), fg="white", bg="#444444")
label_titulo.pack(side="left", padx=10)

btn_style = {
    "font": ("Arial", 16, "bold"),
    "fg": "#FFFFFF",
    "bg": "#444444",
    "activebackground": "#666666",
    "bd": 3,
    "padx": 20,
    "pady": 12,
}

label_palabra = tk.Label(tk_root, text="", font=("Arial", 44, "bold"), fg="#00FF00", bg="#2E2E2E")
label_palabra.pack(pady=20)

btn_traducir = tk.Button(tk_root, text="Ver Significado", command=mostrar_traduccion, **btn_style)
btn_traducir.pack(pady=10)

label_traduccion = tk.Label(tk_root, text="", font=("Arial", 44), fg="#0000FF", bg="#2E2E2E")
label_traduccion.pack(pady=10)

btn_pronunciar = tk.Button(tk_root, text="Escuchar Pronunciación", command=pronunciar, **btn_style)
btn_pronunciar.pack(pady=10)

frame_navegacion = tk.Frame(tk_root, bg="#2E2E2E")
frame_navegacion.pack(pady=20)

btn_anterior = tk.Button(frame_navegacion, text="⏪ Anterior", command=anterior_palabra, **btn_style)
btn_anterior.grid(row=0, column=0, padx=20)

btn_siguiente = tk.Button(frame_navegacion, text="Siguiente ⏩", command=siguiente_palabra, **btn_style)
btn_siguiente.grid(row=0, column=1, padx=20)

btn_info = tk.Button(tk_root, text="ℹ️ Info", font=("Arial", 12, "bold"), command=mostrar_info, bg="#555555", fg="white", bd=2)
btn_info.place(relx=1.0, rely=1.0, anchor="se", x=-10, y=-10)

mostrar_palabra()
tk_root.mainloop()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import os

#Funcion para obtener el path absoluto
def resource_path(relative_path):
 try:
# PyInstaller creates a temp folder and stores path in _MEIPASS 
   base_path = sys._MEIPASS
 except Exception:
   base_path = os.path.abspath(".")
 return os.path.join(base_path, relative_path)



c:\Users\jrgui\Documents\GitHub\Linguo-1.0\Linguo 2.0\logo\logo.ico
